# 1. 라이브러리 로드

In [ ]:
#!pip install shap
#!pip install lime
#!pip install klib

In [ ]:
from time import time
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import make_pipeline

from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

import pandas as pd 
import numpy as np 
#from sklearn.datasets import load_boston 
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.inspection import plot_partial_dependence 
from sklearn.inspection import partial_dependence 
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import shap
import lime
import lime.lime_tabular
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore')

# EDA
import klib
import seaborn as sns

# 2. 데이터셋 로드

In [ ]:
#boston = load_boston() 
#boston.keys() 
# dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename']) 

#X = pd.DataFrame(boston['data'], columns=boston['feature_names']) 
#y = pd.DataFrame(boston['target'], columns=['target'])
X

# 3. 탐색적 데이터 분석

### 결측치 확인

In [ ]:
X.isnull().sum()

In [ ]:
y.isnull().sum()

### Target의 분포 확인

In [ ]:
# 수치형 속성 파악(시각화)
klib.dist_plot(y)
plt.show()

### 독립변수의 분포 확인

In [ ]:
# 수치형 속성 파악(시각화)
klib.dist_plot(X)
plt.show()

### 변수 간 상관성 확인

In [ ]:
sns.heatmap(X.corr(), cmap='RdYlGn', linewidths=0.2) #titanic.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(12,10)
plt.show()

## train, test 나누기

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# 4. 모델링 및 모델 해석

In [ ]:
model_rf = RandomForestRegressor(n_estimators=100, random_state=0) 
model_rf.fit(X_train, y_train)

In [ ]:
y_pred = model_rf.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)**(0.5)
mse

# 4.1 PDP(Partial Dependence Plot)
- 모델에 상관없이 적용 가능
- 특정 feature와 target 사이의 관계를 알기 위함, 특정 feature의 값을 변화시키며 target 값의 변화를 관찰

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
plot_partial_dependence(estimator=model_rf, X=X_test, features=['LSTAT'], 
                        grid_resolution=round(X_test.shape[0]*0.1), percentiles=(0, 1), 
                        kind='average', method='brute',
                        ax=ax);

In [ ]:
fig = plt.figure(figsize=(6,6))


features = ('CRIM', 'ZN')
pdp, axes = partial_dependence(model_rf, X_test, features=features, grid_resolution=20).values()

XX, YY = np.meshgrid(axes[0], axes[1])
Z = pdp[0].T
ax = Axes3D(fig)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1, cmap=plt.cm.BuPu, edgecolor='k')

ax.set_xlabel(features[0])
ax.set_ylabel(features[1])
ax.set_zlabel('Partial dependence')
               
# pretty init view
ax.view_init(elev=22, azim=122)
plt.colorbar(surf)
plt.suptitle('Partial dependence of house value \n'
             'with RandomForest')
plt.subplots_adjust(top=0.9)
               
plt.show()

# 4.2 ICE (Individual Conditional Expectation Plot)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
plot_partial_dependence(estimator=model_rf, X=X_test, features=['LSTAT'], 
                        grid_resolution=round(X_train.shape[0]*0.1), percentiles=(0, 1), 
                        kind='individual', method='brute',
                        ax=ax)

# 4.3 LIME

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values.tolist(),
                                                  class_names=['MEDV'], verbose=True, mode='regression')

In [ ]:
#5th instance
j = 5
exp = explainer.explain_instance(X_test.values[j], model_rf.predict, num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
#10th instance
j = 10
exp = explainer.explain_instance(X_test.values[j], model_rf.predict, num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
#15th instance
j = 15
exp = explainer.explain_instance(X_test.values[j], model_rf.predict, num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
#20th instance
j = 20
exp = explainer.explain_instance(X_test.values[j], model_rf.predict, num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

In [ ]:
#25th instance
j = 25
exp = explainer.explain_instance(X_test.values[j], model_rf.predict, num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True)

In [ ]:
exp.as_list()

# 4.4 Shap

In [ ]:
shap.initjs()
explainer = shap.Explainer(model_rf, X_test)
shap_values = explainer(X_train)

### Feture Importance Plot

In [ ]:
shap.plots.bar(shap_values)

### Summary Plot

In [ ]:
shap.summary_plot(shap_values)

### Waterfall Plot

In [ ]:
shap.plots.waterfall(shap_values[201])

### Force Plot

In [ ]:
shap.force_plot(shap_values[1])

In [ ]:
shap.plots.force(shap_values[0:1000, :])

# 5. 결과 해석

In [ ]:
# 여러 XAI 결과를 확인하여 해석